Connected to Python 3.10.11

### Load libraries

In [ ]:
pip install statsmodels

In [1]:
import sys
import os
import itertools
import matplotlib.pyplot as plt
import pandas as pd

# Set paths
notebook_dir = os.getcwd()
src_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\src"
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import add_src_to_path
add_src_to_path()

from data_split import split_time_series_by_river
from rf_model import train_and_apply_rf_with_tuning
from plot_predictions import plot_predictions_by_river
from plot_predictions import plot_actual_vs_predicted

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

# Experiment parameters
river_systems = ["Fraser River", "Bristol Bay", "Columbia River"]
top_k_options = [6, 10, 20]
add_feat_options = [True, False]
arima_options = [True, False]

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_path = os.path.join(project_root, 'data', 'Combined_FeatureSet_For_Model.csv')

model_list = ["RF", "GBRT", "XGB", "LR", "PR"]
all_experiments = []

for river_system, top_k_features, add_additional_features, use_arima_on_top in itertools.product(
    river_systems, top_k_options, add_feat_options, arima_options
):
    print(f"\n🧪 Running: {river_system} | TopK={top_k_features} | ExtraFeat={add_additional_features} | ARIMA={use_arima_on_top}")

    combined_df = pd.read_csv(data_path)
    combined_df = combined_df[combined_df["System"] == river_system]

    if add_additional_features:
        features_to_lag = [
            'Total_Returns', 'AgeClass_0.1', 'AgeClass_0.2', 'AgeClass_0.3', 'AgeClass_0.4', 'AgeClass_0.5',
            'AgeClass_1.1', 'AgeClass_1.2', 'AgeClass_1.3', 'AgeClass_1.4', 'AgeClass_1.5',
            'AgeClass_2.1', 'AgeClass_2.2', 'AgeClass_2.3', 'AgeClass_2.4',
            'AgeClass_3.1', 'AgeClass_3.2', 'AgeClass_3.3', 'AgeClass_3.4',
            'Total_Returns_NextYear', 'Pacea_ALPI_Anomaly', 'npi_mean_NovMar', 'oni_mean_DecFeb',
            'npgo_mean_DecFeb', 'ao_mean_DecMar', 'pdo_mean_DecMar', 'pdo_mean_MaySep'
        ]
        for feat in features_to_lag:
            for lag in [1, 2, 3, 4, 5]:
                combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)

    combined_df = combined_df.dropna(axis=1, how='any').dropna()

    train_df, test_df = split_time_series_by_river(
        combined_df,
        time_column="Year",
        group_columns=["System", "River"],
        test_fraction=0.2,
        gap_years=0
    )

    train_df["River_Name"] = train_df["River"]
    test_df["River_Name"] = test_df["River"]

    train_df_encoded = pd.get_dummies(train_df, columns=["River"], prefix="River")
    test_df_encoded = pd.get_dummies(test_df, columns=["River"], prefix="River")

    for model_name in model_list:
        try:
            results = train_and_apply_rf_with_tuning(
                model=model_name,
                train_df=train_df_encoded,
                test_df=test_df_encoded,
                topk_feat=top_k_features
            )

            # ✅ Apply ARIMA correction if selected
            if use_arima_on_top:
                residuals = results["Timeline_train"]["Actual"] - results["Timeline_train"]["Predicted"]
                residuals_series = pd.Series(residuals.values, index=results["Timeline_train"]["Year"])

                arima_model = ARIMA(residuals_series, order=(1, 0, 0))
                arima_fit = arima_model.fit()

                residual_forecast = arima_fit.forecast(steps=len(results["Timeline_test"]["Predicted"]))
                hybrid_pred = results["Timeline_test"]["Predicted"] + residual_forecast.values

                results["Timeline_test"]["Predicted"] = hybrid_pred

                # Optionally update metrics
                r2 = r2_score(results["Timeline_test"]["Actual"], hybrid_pred)
                mse = mean_squared_error(results["Timeline_test"]["Actual"], hybrid_pred)
                mape = mean_absolute_percentage_error(results["Timeline_test"]["Actual"], hybrid_pred) * 100

                # Also update metrics per river (recalculate)
                grouped = results["Timeline_test"].groupby("River_Name")
                metrics_by_river_test = grouped.apply(
                    lambda g: pd.Series({
                        "R2": r2_score(g["Actual"], g["Predicted"]),
                        "MSE": mean_squared_error(g["Actual"], g["Predicted"]),
                        "MAPE": mean_absolute_percentage_error(g["Actual"], g["Predicted"]) * 100
                    })
                ).reset_index()

                results["Metrics_by_River_Test"] = metrics_by_river_test

            # 📊 Prepare final output
            test_df_ = results['Metrics_by_River_Test'].copy()
            train_df_ = results['Metrics_by_River_Train'].copy()

            test_df_ = test_df_.rename(columns={"R2": "R2_Test", "MSE": "MSE_Test", "MAPE": "MAPE_Test"})
            train_df_ = train_df_.rename(columns={"R2": "R2_Train", "MSE": "MSE_Train", "MAPE": "MAPE_Train"})

            merged_df = pd.merge(test_df_, train_df_, on="River_Name", how="outer")
            merged_df.insert(0, "Model", model_name)

            river_system_lookup = pd.concat([train_df_encoded, test_df_encoded])["River_Name"].drop_duplicates()
            river_system_lookup = pd.merge(
                river_system_lookup.to_frame(),
                combined_df[["River", "System"]].drop_duplicates(),
                left_on="River_Name",
                right_on="River",
                how="left"
            ).drop(columns="River")

            merged_df = pd.merge(merged_df, river_system_lookup, on="River_Name", how="left")

            merged_df["Selected_System"] = river_system
            merged_df["TopK_Features"] = top_k_features
            merged_df["Additional_Features_Used"] = add_additional_features
            merged_df["ARIMA_Enabled"] = use_arima_on_top

            all_experiments.append(merged_df)

        except Exception as e:
            print(f"❌ Error with model {model_name}: {e}")

# 📝 Save final results
final_df = pd.concat(all_experiments, ignore_index=True)
output_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\results.csv"
final_df.to_csv(output_path, index=False)

print(f"\n✅ All experiment results saved to:\n{output_path}")



🧪 Running: Fraser River | TopK=6 | ExtraFeat=True | ARIMA=True
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_29848\619836447.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_29848\619836447.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_29848\619836447.py:55: PerformanceWarning: DataFrame 

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
RF R2: 0.41
RF MSE: 365025695484.21
RF MAPE: 477.37
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:145: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:149: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'max_iter': 200, 'min_samples_leaf': 1}
GBRT R2: 0.21
GBRT MSE: 486627008236.27
GBRT MAPE: 442.80
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


...

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
XGB R2: 0.43
XGB MSE: 354220998416.79
XGB MAPE: 382.47
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')
Best Parameters: {'fit_intercept': True, 'positive': False}
LR R2: 0.34
LR MSE: 408886514095.51
LR MAPE: 1753.96
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:145: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_test = results_df.groupby("System", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:149: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_N

No parameter tuning for this model.
PR R2: 0.16
PR MSE: 518184352425.38
PR MAPE: 1269.42

🧪 Running: Fraser River | TopK=6 | ExtraFeat=False | ARIMA=False
Selected features:
Index(['Total_Returns', 'AgeClass_0.2', 'AgeClass_1.1', 'River_Chilko',
       'River_Quesnel', 'River_Raft'],
      dtype='object')


c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:149: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  river_metrics_test = results_df.groupby("River_Name", group_keys=False).apply(
c:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\rf_model.py:153: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  system_metrics_train = train_results_df.group

KeyboardInterrupt: 

### Optional: Fit ARIMA model on residuals (only works if only one river selected so far)

In [ ]:
if True:    
    residuals = results["Timeline_train"]["Actual"] - results["Timeline_train"]["Predicted"]

    from statsmodels.tsa.arima.model import ARIMA
    residuals_series = pd.Series(residuals.values, index=results["Timeline_train"]["Year"])
    arima_model = ARIMA(residuals_series, order=(1,0,0))  # You may want to auto-tune this
    arima_fit = arima_model.fit()
    residual_forecast = arima_fit.forecast(steps=len(results["Timeline_test"]["Predicted"]))

    hybrid_pred = results["Timeline_test"]["Predicted"] + residual_forecast.values
    from sklearn.metrics import mean_squared_error, r2_score
    r2 = r2_score(results["Timeline_test"]["Actual"], hybrid_pred)
    print(r2)

    results["Timeline_test"]["Predicted"] = hybrid_pred

In [ ]:
sorted_items = sorted(results["Feature_Importances"].items(), key=lambda x: x[1], reverse=True)
features, importances = zip(*sorted_items)

# Plotting
plt.figure(figsize=(10, 6))
plt.barh(features, importances, color='skyblue')
plt.xlabel('Feature Importance')
plt.title('Feature Importances')
plt.gca().invert_yaxis()  # Most important on top
plt.tight_layout()
plt.show()